In [19]:
import warnings, numpy, os

import pandas as pd

from langdetect import detect

from google.cloud import translate_v2 as translate

In [20]:
posts_path = 'CSVs/linkedin_posts/unlabeled.csv'
posts_translated_path = 'CSVs/linkedin_posts/unlabeled_translated.csv'

In [21]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"Credentials/dna-interno.json"

In [22]:
translate_client = translate.Client()

translate_client.translate('This is a text that i expect bee in portuguese', target_language='pt')

{'translatedText': 'Este é um texto que espero estar em português',
 'detectedSourceLanguage': 'en',
 'input': 'This is a text that i expect bee in portuguese'}

In [23]:
def get_posts_csv():
    try: return pd.read_csv(posts_path)
    except: return []
    
get_posts_csv()

,id,company,post_title,post_date,post_text,post_likes,post_comments,post_video_views,post_hashtags,origin,Unnamed: 10
0,95,softexnacional,NaN,2020-03-05,'Startups e PMEs: a Comissão Europeia está con...,3,0,0,NaN,linkedin,NaN
1,167,softexnacional,NaN,2020-03-05,'O Programa IA2 MCTIC surgiu no intuito de pro...,10,0,0,NaN,linkedin,NaN
2,168,softexnacional,NaN,2020-03-05,"'Nosso Programa de Certificações, MPS.BR, tamb...",7,0,0,NaN,linkedin,NaN
3,170,softexnacional,NaN,2020-03-05,'Por pautarmos sempre nossas ações na #coletiv...,24,0,0,"#coletividade, #respeito, #fiqueemcasa'",linkedin,NaN
4,171,softexnacional,NaN,2020-03-05,'Desafio Covid19: https://lnkd.in/eUSd67k.Uma ...,9,0,0,NaN,linkedin,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1447,1,globant,NaN,2021-03-05,NaN,0,0,0,NaN,linkedin,NaN
1448,1,softexnacional,NaN,2021-03-05,NaN,0,0,0,NaN,linkedin,NaN
1449,1,bairesdev,NaN,2021-03-05,'We are looking for Python Developers to join ...,16,0,0,"#NowHiring, #Openings, #JobVacancy'",linkedin,NaN
1450,2,bairesdev,NaN,2021-03-05,"'Gender diversity matters. At BairesDev, we pr...",26,1,0,"#GenderDiversity, #GenderEquity'",linkedin,NaN


In [24]:
def detect_languages():
    df = get_posts_csv()
    for i, post in df.iterrows():
        df.loc[i, 'language'] = detect(str(post['post_text']))
        
    return df

In [25]:
detect_languages().to_csv(posts_path, index=False)

In [26]:
get_posts_csv()

,id,company,post_title,post_date,post_text,post_likes,post_comments,post_video_views,post_hashtags,origin,Unnamed: 10,language
0,95,softexnacional,NaN,2020-03-05,'Startups e PMEs: a Comissão Europeia está con...,3,0,0,NaN,linkedin,NaN,pt
1,167,softexnacional,NaN,2020-03-05,'O Programa IA2 MCTIC surgiu no intuito de pro...,10,0,0,NaN,linkedin,NaN,pt
2,168,softexnacional,NaN,2020-03-05,"'Nosso Programa de Certificações, MPS.BR, tamb...",7,0,0,NaN,linkedin,NaN,pt
3,170,softexnacional,NaN,2020-03-05,'Por pautarmos sempre nossas ações na #coletiv...,24,0,0,"#coletividade, #respeito, #fiqueemcasa'",linkedin,NaN,pt
4,171,softexnacional,NaN,2020-03-05,'Desafio Covid19: https://lnkd.in/eUSd67k.Uma ...,9,0,0,NaN,linkedin,NaN,pt
...,...,...,...,...,...,...,...,...,...,...,...,...
1447,1,globant,NaN,2021-03-05,NaN,0,0,0,NaN,linkedin,NaN,tl
1448,1,softexnacional,NaN,2021-03-05,NaN,0,0,0,NaN,linkedin,NaN,tl
1449,1,bairesdev,NaN,2021-03-05,'We are looking for Python Developers to join ...,16,0,0,"#NowHiring, #Openings, #JobVacancy'",linkedin,NaN,en
1450,2,bairesdev,NaN,2021-03-05,"'Gender diversity matters. At BairesDev, we pr...",26,1,0,"#GenderDiversity, #GenderEquity'",linkedin,NaN,en


In [27]:
get_posts_csv()['language'].value_counts()

en    1056
pt     230
es     158
tl       3
it       3
ca       2
Name: language, dtype: int64

In [54]:
def convert_to_pt():
    df = get_posts_csv()
    total_chars = 0
    for i, post in df.iterrows():
        
        text = str(post['post_text'])
        lg = str(post['language'])
        text_pt = str(post['post_text_pt'])
        
        if text_pt != 'nan': continue
        
        if lg == 'pt':
            df.loc[i, 'post_text_pt'] = text
            print('Texto já em portugues')
        else:
            total_chars += len(text)
            result = translate_client.translate(text, target_language='pt')
            df.loc[i, 'post_text_pt'] = result['translatedText']
            
            print('Total chars:', total_chars)
        
    return df

In [55]:
df_translated = convert_to_pt()

Total chars: 199
Total chars: 391
Total chars: 607
Total chars: 778
Total chars: 931
Total chars: 1097
Total chars: 1345
Total chars: 1596
Total chars: 1792
Total chars: 1978
Total chars: 2257
Total chars: 2436
Total chars: 2656
Total chars: 2837
Total chars: 3258
Total chars: 3427
Total chars: 3713
Total chars: 3866
Total chars: 4037
Total chars: 4194
Total chars: 4430
Total chars: 4669
Total chars: 4899
Total chars: 5092
Total chars: 5311
Total chars: 5459
Total chars: 5611
Total chars: 5716
Total chars: 5897
Total chars: 6068
Total chars: 6219
Total chars: 6358
Total chars: 6509
Total chars: 6702
Total chars: 6905
Total chars: 7040
Total chars: 7201
Total chars: 7368
Total chars: 7556
Total chars: 7780
Total chars: 7969
Total chars: 8162
Total chars: 8285
Total chars: 8460
Total chars: 8643
Total chars: 8840
Total chars: 9059
Total chars: 9218
Total chars: 9398
Total chars: 9547
Total chars: 9640
Total chars: 9865
Total chars: 10149
Total chars: 10343
Total chars: 10513
Total chars:

In [57]:
df_translated

,id,company,post_title,post_date,post_text,post_likes,post_comments,post_video_views,post_hashtags,origin,language,post_text_pt
0,72,bairesdev,NaN,2020-04-08,"Our CTO, Pablo Azorin talks about how for Bair...",276,5,0,"#WorkFromHome, #HomeOffice, #BusinessAsUsualAtBD'",linkedin,en,"O nosso CTO, Pablo Azorin, fala sobre como par..."
1,119,bairesdev,NaN,2020-04-08,Nacho De Marco posted a new article analyzing ...,70,1,0,"#Business, #Future'",linkedin,en,Nacho De Marco publicou um novo artigo analisa...
2,215,bairesdev,NaN,2020-04-08,"""Nacho De Marco, BairesDev's CEO, shared some ...",221,6,0,"#WorkFromHome, #HomeOffice, #BusinessAsUsualAtBD""",linkedin,en,"&quot;Nacho De Marco, CEO da BairesDev, compar..."
3,279,bairesdev,NaN,2020-04-08,"""Pablo Chamorro, BairesDev's Chief Revenue Off...",117,0,0,"#Motivation, #MotivationAtWork""",linkedin,en,"&quot;Pablo Chamorro, Chief Revenue Officer da..."
4,288,bairesdev,NaN,2020-04-08,Check out our new white paper on how AI can ad...,56,0,0,"#ArtificialIntelligence, #AI, #Business'",linkedin,en,Confira nosso novo white paper sobre como a IA...
...,...,...,...,...,...,...,...,...,...,...,...,...
1451,3,softexnacional,NaN,2021-03-02,"'É amanhã! Dia 03/03 às 10h, evento de lançame...",7,1,0,NaN,linkedin,pt,"'É amanhã! Dia 03/03 às 10h, evento de lançame..."
1452,7,softexnacional,NaN,2021-03-02,'#MinutoSoftex.🚀Vamos promover nesta quarta-fe...,12,0,0,"#MinutoSoftex.🚀Vamos, #UnboxingInternational,",linkedin,pt,'#MinutoSoftex.🚀Vamos promover nesta quarta-fe...
1453,2,softexnacional,NaN,2021-03-03,'Notícias da Semana! https://lnkd.in/dDe4a-V ....,12,0,0,"#NewsSoftex, #NotíciasDaSemana",linkedin,pt,'Notícias da Semana! https://lnkd.in/dDe4a-V ....
1454,0,softexnacional,NaN,2021-03-04,'🎧🎙 Episódio #01 #TIcomElas no ar: softex.br/s...,16,0,0,"#01, #TIcomElas",linkedin,pt,'🎧🎙 Episódio #01 #TIcomElas no ar: softex.br/s...


In [58]:
df_translated.to_csv(posts_translated_path, index=False)